# Customer Lifetime Value Analysis

In [8]:
import sqlite3
import pandas as pd
from matplotlib import pyplot as plt
conn = sqlite3.connect('mock_resq.db')
cursor = conn.cursor()

### Average purchase value

In [46]:
avgPurchaseValue = """
SELECT 
    SUM(order_sales) / COUNT(order_id) AS avgPurchaseValue
FROM 
    master_orders_table;
"""
avgPurchaseValue = conn.execute(avgPurchaseValue).fetchall()[0][0]
avgPurchaseValue

770

### Average purchase frequency 

In [57]:
avgPurchaseFrequency = """
SELECT 
    COUNT(order_id)  / COUNT(DISTINCT user_id) AS avgPurchaseFrequency
FROM 
    master_orders_table;
"""
avgPurchaseFrequency = conn.execute(avgPurchaseFrequency).fetchall()[0][0]
avgPurchaseFrequency

2

### Customer Retention Rate

In [58]:
cust_retention_rate = """
WITH customer_orders AS (
    SELECT 
        user_id,
        COUNT(order_id) AS totalOrders
    FROM 
        master_orders_table
    GROUP BY 
        user_id
)
SELECT 
    (COUNT(CASE WHEN totalOrders > 1 THEN 1 END) * 1.0 / COUNT(*)) AS retentionRate
FROM 
    customer_orders;
"""
cust_retention_rate = conn.execute(cust_retention_rate).fetchall()[0][0]
cust_retention_rate

0.4605788585156808

### Customer Life Time Estimate

In [59]:
cust_life_time = 1.0 / (1 - cust_retention_rate) 
cust_life_time

1.8538390936037676

### Customer Life Time Value

In [60]:
avgPurchaseValue*avgPurchaseFrequency*cust_life_time

2854.912204149802

### Complete Query

In [61]:
query = """
WITH avg_purchase_value AS (
    SELECT 
        SUM(order_sales) / COUNT(order_id) AS avgPurchaseValue
    FROM 
        master_orders_table
),
avg_purchase_frequency AS (
    SELECT 
        COUNT(order_id)  / COUNT(DISTINCT user_id) AS avgPurchaseFrequency
    FROM 
        master_orders_table
),
retention_rate AS (
    WITH customer_orders AS (
        SELECT 
            user_id,
            COUNT(order_id) AS totalOrders
        FROM 
            master_orders_table
        GROUP BY 
            user_id
    )
    SELECT 
        (COUNT(CASE WHEN totalOrders > 1 THEN 1 END) * 1.0 / COUNT(*)) AS retentionRate
    FROM 
        customer_orders
),
customer_lifetime AS (
    SELECT 
        1.0 / (1 - retentionRate) AS lifetime
    FROM 
        retention_rate
)
SELECT 
    avg_purchase_value.avgPurchaseValue * 
    avg_purchase_frequency.avgPurchaseFrequency * 
    customer_lifetime.lifetime AS customerLifetimeValue
FROM 
    avg_purchase_value, avg_purchase_frequency, customer_lifetime;
"""

In [62]:
cust_lifetime_value = conn.execute(query).fetchall()[0][0]
cust_lifetime_value

2854.912204149802